In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy
import json
import math
import datetime
from sqlalchemy.ext.declarative import declarative_base
from scipy import integrate

In [ ]:
FIGSIZE = (40, 20)
FONTSIZE = 20

PRODUCTION_COLOR = "mediumspringgreen"
CONSUMPTION_COLOR = "coral"
NET_COLOR = "grey"

with open("../localconfig.json", "r") as f:
	config = json.load(f)



# Load solar data from files.

In [ ]:
FILES = [ "../local-solardata.csv", "../local-solardata-2022-05-12.csv" ]

solardf = None

for f in FILES:


	df = pd.read_csv(f,
					header=None,
					names=['datetime', 'consumption', 'production'],
					index_col=0,
					parse_dates=[0])
	if solardf is not None:
		solardf = solardf.append(df)
	else:
		solardf = df

solardf.sort_index(inplace=True)
		

# Load solar production data from a database

In [ ]:
connectionstr = f"mysql+pymysql://{config['dbuser']}:{config['dbpass']}@{config['dbhost']}/{config['dbname']}?charset=utf8mb4"
engine = sqlalchemy.create_engine(connectionstr)
engine.connect()

In [ ]:
data_to = datetime.datetime.now()
data_from = data_to - datetime.timedelta(days=7)
data_from_str = data_from.strftime("%Y-%m-%d")
data_to_str = data_to.strftime("%Y-%m-%d")
#data_from_str = '2022-05-18'
#data_to_str = '2022-05-19'

#solardf = pd.read_sql_table('production', engine)
sql =  f'''
SELECT datetime, consumption, production FROM production 
	WHERE datetime >= '{data_from_str}' and datetime < '{data_to_str}'
	ORDER by datetime asc
'''
solardf = pd.read_sql(sql, engine, index_col='datetime')

# Resample data
We may wish to resample data.  Reasons to do this maybe
 * To fill gaps with reasonable values
 * Decrease the number of data points for faster processing
  
`resample()` will resample at our chosen time interval.  Where there is no data, the value is NaN.  There are a couple options
for handling NaN:

 * `dropna()`: will drop these cells.  If graphing, the effect is a gap
 * `fillna()`: will fill these cells with a value.  But what?
 * `bfill()` or `ffill()`: fill backwards from the next value or forward from the previous.  If the gap is small, this is reasonable but if the gap is large it is inaccurate.
 * `interpolate()`:  Will interpolate between the values preceeding the gap and followign the gap.  This may produce the best estimate.

Sample Code:

	df_noop = solardf.resample('1Min').mean()
	df_dropna = solardf.resample('1Min').mean().dropna()
	df_fillna = solardf.resample('1Min').mean().fillna(0)
	df_ffill = solardf.resample('1Min').mean().ffill()
	df_interpolate = solardf.resample('1Min').mean().interpolate(method='linear', axis=0)


In [ ]:
solardf.head()


In [ ]:
plt.rcParams.update({'font.size': FONTSIZE})
fig = plt.figure(figsize = FIGSIZE)

# Resample and interpolate to fill gaps in data with reasonable value.
df = solardf.resample('1Min').mean().interpolate(method='linear', axis=0)

# Smooth data with rolling average and convert to kiloWatts.
minutes_rolling = 20
smoothed_consumption = -df['consumption'].rolling(minutes_rolling).mean().dropna() / 1000
smoothed_production = df['production'].rolling(minutes_rolling).mean().dropna() / 1000
net = smoothed_production + smoothed_consumption


plt.plot(smoothed_consumption.index, smoothed_consumption, CONSUMPTION_COLOR, label="Consumption")
plt.plot(smoothed_production.index, smoothed_production, PRODUCTION_COLOR, label="Production")
plt.plot(net.index, net, NET_COLOR, label="Net")

# Calculate a max value for the y-axis
max_value = math.ceil(max(-smoothed_consumption.max(), smoothed_production.max()))
step = 0.5
plt.yticks(np.arange(-max_value, max_value+step, step))
plt.ylabel("kW")

plt.grid(alpha =.6, linestyle ='--') 
plt.legend()
plt.title("Solar Production")

# Plot historic production and consumption

In [ ]:
#
# Copy the original dataframe
intdf = solardf.copy()

# Add back data point time as a column so we can calculate on it.
intdf['datetime'] = intdf.index

# Calculate the cumulative seconds since first data point.
#     	.diff() calculates difference from previoius row.
# 		.dt.total_seconds()  reats the value as datetime and gets total seconds
# 		fillna(0) ensures there are no missing data elements.
# 		cumsum() calculates cumulative sum at each row.
intdf['cumsec'] = intdf['datetime'].diff().dt.total_seconds().fillna(0).cumsum()

In [ ]:
intdf

In [ ]:
#
# Function to integrate over in groups.
#
def calculate_cumulative(groups, divisor):
	# Arrays to hold values for each column
	cons = []
	prod = []
	index = []

	for name, group in groups:
		#print (group)
		cwh = integrate.trapz(group['consumption'], x=group['cumsec']) / divisor
		pwh = integrate.trapz(group['production'], x=group['cumsec']) / divisor
		cons.append(cwh)
		prod.append(pwh)
		index.append(name)

	return pd.DataFrame({'consumption': cons, 'production': prod}, index=index)


In [ ]:
group_by_day = intdf.groupby(pd.Grouper(axis=0, freq='D'))
daily = calculate_cumulative(group_by_day, 3600*1000)
daily

In [ ]:
np.arange(-60,65, 5)

In [ ]:
x = np.arange(len(daily.index))
width = 0.85
fig, ax = plt.subplots(figsize = (20,10))

rects1 = ax.bar(x, -daily['consumption'], width, label='Consumption', color=CONSUMPTION_COLOR)
rects2 = ax.bar(x, daily['production'], width, label='Production', color=PRODUCTION_COLOR)
rects3 = ax.bar(x, daily['production']-daily['consumption'], width, label='Net', color=NET_COLOR)
ax.set_ylabel('kWh')
ax.set_title('Daily Electric consumption and production')
ax.set_yticks(np.arange(-60,65, 5))
ax
ax.set_xticks(x, daily.index.map(lambda x: x.strftime('%Y-%m-%d')))
ax.legend()
ax.grid(alpha =.6, linestyle ='--')


In [ ]:
#
# Plot daily graphs.
# Enphase shows daily data in 15 minute intervals.
group_by_15min = intdf.groupby(pd.Grouper(axis=0, freq='15min'))
at15min = calculate_cumulative(group_by_15min, 3600)
at15min

In [ ]:
# Past 7 days
daily[-7:].sum()

In [ ]:

g['production'].apply(lambda g: integrate.trapz(g, x=g.index))


In [ ]:
g.apply(lambda g: integrate.trapz(g.consumption, ))